# Initial Import

In [7]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

from main import backgammon

## Import Dataset

In [8]:
class BackgammonDataset(Dataset):
    def __init__(self, file_path):
        # Load CSV file
        data = pd.read_csv(file_path, header=None)  # No header in dataset

        # Convert to numpy arrays
        self.X = data.iloc[:, :-1].values.astype(np.float32)  # First 28 columns as input
        self.y = data.iloc[:, -1].values.astype(np.float32)   # Last column as output

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# 2️⃣ Split Dataset into Training (80%) and Testing (20%)
def get_dataloaders(file_path, batch_size=32, test_size=0.2):
    dataset = BackgammonDataset(file_path)

    # Split indices into training and testing
    train_size = int((1 - test_size) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [9]:
class BackgammonNet(nn.Module):
    def __init__(self):
        super(BackgammonNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [10]:
def get_dataloaders(file_path, batch_size=32, test_size=0.2):
    dataset = BackgammonDataset(file_path)

    # Split indices into training and testing
    train_size = int((1 - test_size) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [11]:
from time import sleep
# 3️⃣ Define Neural Network Model
class BackgammonNet(nn.Module):
    def __init__(self):
        super(BackgammonNet, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(28, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            # nn.Dropout(0.3),
            
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

# 4️⃣ Train the Model
def train_model(train_loader, test_loader, epochs=50, lr=0.001):
    model = BackgammonNet()
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Evaluate on test data
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                test_loss += loss.item()
        if epoch % 10 == 9:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss:.4f}, Test Loss: {test_loss:.4f}")
            torch.save(model.state_dict(), f"backgammon_model_{epoch}.pth")
            # sleep(0.1)
            fitness = backgammon(25, "DEEP",epoch)
            print(fitness)
    return model

In [12]:
if __name__ == "__main__":
    file_path = "../Data/Deep/BoardEquity/board_equity_db.txt"  # Update with actual dataset file
    train_loader, test_loader = get_dataloaders(file_path, batch_size=64)
    
    trained_model = train_model(train_loader, test_loader, epochs=100)
    
    # Save the trained model
    torch.save(trained_model.state_dict(), "backgammon_model.pth")
    print("Model training complete and saved.")

Epoch 100/500, Train Loss: 0.6729, Test Loss: 0.2404
([1, 0, 0], 1, [1, 0, 8], 25)
Epoch 200/500, Train Loss: 0.5608, Test Loss: 0.2521
([0, 0, 0], 0, [0, 0, 9], 27)
Epoch 300/500, Train Loss: 0.4203, Test Loss: 0.2336
([0, 0, 0], 0, [0, 0, 9], 27)
Epoch 400/500, Train Loss: 0.3998, Test Loss: 0.2368
([0, 0, 0], 0, [0, 0, 9], 27)
Epoch 500/500, Train Loss: 0.3606, Test Loss: 0.2307
([0, 0, 0], 0, [0, 0, 9], 27)
Model training complete and saved.
